In [4]:
import os
from dotenv import load_dotenv
import google

In [6]:
load_dotenv()

if not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("❌ Erreur: Clé API manquante dans le fichier .env")

print("✅ Gemini API key setup complete.")

✅ Gemini API key setup complete.


In [7]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [8]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)
print("Settings done")

Settings done


In [10]:
research_agent = Agent(
    name = 'researcher',
    model=Gemini(retry_options=retry_config, model='gemini-2.5-flash-lite'),
    instruction='You are a researcher agent, you are only goal is to find two or three informations on the topic provided by the user',
    tools = [google_search],
    output_key='research-findings'
)

In [11]:
summarize_agent = Agent(
    name = 'summarizer',
    model = Gemini(retry_options=retry_config, model='gemini-2.5-flash-lite'),
    instruction = 'Your only goal is to summarize in 100 words the informations on a topic I am going to give to you. Here ar' \
    'e the information : {research-finding}' ,
    output_key= 'summary'
)

In [14]:
root_agent = Agent(
    name = 'root_agent',
    model=Gemini(retry_options=retry_config, model='gemini-2.5-flash-lite'),
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow. \
    1. First, you MUST call the `researcher` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `summarizer` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
tools = [AgentTool(research_agent), AgentTool(summarize_agent)],
)

In [20]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are the latest advancements in quantum computing and what do they mean for AI?"
)


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?
root_agent > The latest advancements in quantum computing are set to revolutionize AI by significantly accelerating AI model training and optimization, enhancing data processing and analysis capabilities, and enabling AI to tackle problems currently beyond the reach of classical computers. This fusion, known as Quantum AI, could lead to more efficient and reliable AI systems, improve natural language processing, and bolster cybersecurity through advanced threat detection. While breakthroughs in quantum error correction and the development of more powerful quantum processors are rapidly progressing, challenges such as qubit stability, coherence times, and the scarcity of specialized talent mean widespread adoption may still be a decade away. Hybrid quantum-classical approaches are expected in the interim, combining the strengths of both technologies."
